In [1]:
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt


In [2]:
x_test = np.load('result_2/X_test.npy')
y_test = np.load('result_2/y_test.npy')
x_test.shape, y_test.shape

((2776, 256, 256, 3), (2776, 5))

In [3]:
import efficientnet.keras as efn
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras import utils, models, layers, optimizers
from keras.models import Model, load_model, Sequential

base_model = efn.EfficientNetB2(weights = 'imagenet', include_top = False, pooling = 'avg', input_shape = (256, 256, 3))
x = base_model.output
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
output = Dense(5, activation = 'sigmoid')(x)
model = Model(base_model.input, output)
#model = multi_gpu_model(resnet50, gpus = None)
model.summary()

Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 32) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [4]:
model.load_weights('result_2/weight_Classification_200612_Adam_effb2.hdf5')

In [5]:
idx = 59
img = x_test[idx]
img = tf.cast(np.expand_dims(img, axis=0), tf.float32)
img.shape

TensorShape([1, 256, 256, 3])

In [21]:
from keras import backend as K 



def get_gradient(model, img):

    inp = model.input                                           # input placeholder
    outputs = [layer.output for layer in model.layers]        # all layer outputs except first (input) layer
    functor = K.function([inp, K.learning_phase()], outputs )   # evaluation function

    layer_outputs = functor([img, 1])
    layer_name = np.array([layer.__class__.__name__ for layer in model.layers])
    layer_mask = np.where('Conv2D'==layer_name)

    conv_outputs=[]
    for i in layer_mask[0]:
        conv_out = layer_outputs[i]
        conv_out = tf.convert_to_tensor(conv_out, np.float32)
        conv_outputs.append(conv_out)


    model_outputs = tf.convert_to_tensor(layer_outputs[-1], np.float32)
    print(model_outputs)
    print(len(conv_outputs))
    
    with tf.GradientTape() as tape:
        tape.watch(model_outputs)
        tape.watch(conv_outputs)
        
        gradients = tape.gradient(model_outputs, conv_outputs)  # record     
        print(gradients)
    return gradients

In [22]:
gradients = get_gradient(model, img)

tf.Tensor([[0.02353823 0.37069273 0.07974699 0.25527543 0.2925865 ]], shape=(1, 5), dtype=float32)
92
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


In [20]:
gradients

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]